# 로봇 인공지능 만들기 Assignment 5

기계항공공학부 2013-11815 이승섭

## Problem 1.
2차원 공간에 임의의 지형 지물이 주어졌을 때, 센서가 지형지물과의 거리만 또는 거리와 각도를 준다고 할 때, 로봇 위치를 추정하는 Particle Filter 프로그램을 작성하라.

In [ ]:
# Import relevant libraries